# Notebook based code
You can write code in python in the notebook, ship things from the notebook and get respones, outputs, logs.

## Challenges today
1. You cannot run "nested" `dynamic` etc in a notebook in V1
2. Seeing logs etc and getting the right version is hard

## Things to note!
1. Simple init (we will make this magical soon for serverless etc)
2. asyncio is native in a notebook
3. nested calls square calls hello
4. Observe phases
5. Observe logs

In [1]:
import flyte

In [ ]:
flyte.init(
    endpoint="https://union.example.com",
    org="example_org",
    project="example_project",
    domain="development",
)

In [3]:
env = flyte.TaskEnvironment(
    name="hello_world",
    image=flyte.Image.auto(),
)

In [4]:
import asyncio
from typing import List


@env.task
async def say_hello(data: str, lt: List[int]) -> str:
    print(f"Hello, world! - {flyte.ctx().action}")
    return f"Hello {data} {lt}"


@env.task
async def square(i: int = 3) -> int:
    print(flyte.ctx().action)
    return i * i


@env.task
async def say_hello_nested(data: str = "default string") -> str:
    print(f"Hello, nested! - {flyte.ctx().action}")
    coros = []
    for i in range(100):
        coros.append(square(i=i))

    vals = await asyncio.gather(*coros)
    return await say_hello(data=data, lt=vals)

In [5]:
run = await flyte.run.aio(say_hello_nested, data="jupyter says hello 2!")

09:07:01.266120 INFO     _deploy.py:102 - Building Image for environment hello_world, image: Image(base_image='python:3.12-slim-bookworm', dockerfile=None,     
                         registry='ghcr.io/unionai-oss', name='flyte', platform=('linux/amd64', 'linux/arm64'), tag='py3.12-v0.2.0b4.dev6-g17ff4f7',            
                         python_version=(3, 12), _identifier_override='auto', is_final=False, _layers=(_DockerLines(LABEL                                       
                         org.opencontainers.image.authors='Union.AI <sales@union.ai>'='LABEL                                                                    
                         org.opencontainers.image.source=https://github.com/unionai/unionv2'), Env(env_vars=(('VIRTUAL_ENV', '/opt/venv'), ('PATH',             
                         '/opt/venv/bin:$PATH'), ('PYTHONPATH', '/root'), ('UV_LINK_MODE', 'copy'))), AptPackages(build-essential='ca-certificates'),           
                         PipPackages(packages=('kubernetes', 'msgpack', 'mashumaro')), CopyConfig(1, PosixPath('/Users/ketanumare/src/unionv2/dist'),           
                         _compute_identifier=<function Image.with_local_v2.<locals>.<lambda> at 0x106642020>),                                                  
                         Commands('--mount=type=cache,sharing=locked,mode=0777,target=/root/.cache/uv,id=uv --mount=from=uv,source=/uv,target=/usr/bin/uv       
                         --mount=source=dist,target=/dist,type=bind uv pip install --python $VIRTUALENV $(ls /dist/*whl)')))

09:07:01.272238 INFO     _deploy.py:88 - Building image flyte for environment hello_world

I0000 00:00:1749053221.292629 1652680 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


09:07:07.833418 INFO     image_builder.py:196 - Image ghcr.io/unionai-oss/flyte:py3.12-v0.2.0b4.dev6-g17ff4f7 already exists in registry. Skipping build.

09:07:07.840159 INFO     _deploy.py:111 - Built Image for environment hello_world, image: ghcr.io/unionai-oss/flyte:py3.12-v0.2.0b4.dev6-g17ff4f7

In [ ]:
await run.wait.aio()

Output()

In [23]:
run.name

'wn6qmm897qs8nb9tmhwc'